# Loading data from MySQL database

Sometimes we need to query data directly from a SQL database using Python and mysql.connector is useful for doing that. I pick  mysql.connector even though there are other packages out there that do the same job because mysql.connector works well with the Python Python 3.6.5. For SQL database, I use MySQL.

## Import modules

In [28]:
import pandas as pd
import mysql.connector

## Establish a connection

After loading the modules needed, we need to establish a connection between Python and MySQL database. Replace the username, password, host (usually is localhost) with your own information and replace the database with the one you want to query.

In [30]:
cnx = mysql.connector.connect(user='root', password='BerrybarroW13$',
                              host='localhost',
                              database='country_club')

# Query

Now we are set to write our first query using Pandas' read_sql function.

In [33]:
df = pd.read_sql('SELECT * FROM Facilities', con=cnx)

In [34]:
df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


****
## MySQL exercise

Let's try to use the file country_club database and answer the below questions:

1. Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.
2. How many facilities do not charge a fee to members?
3. How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.
4. How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.
5. How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than 100? Return the name and monthly maintenance of the facilities in question.
6. You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.
7. How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.
8. How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than 30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.
9. This time, produce the same result as in Q8, but using a subquery.
10. Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

## Q1. Name of facilities that charge a fee to members

In [86]:
query1 = 'SELECT DISTINCT name FROM Facilities WHERE membercost > 0'

In [87]:
df1 = pd.read_sql(query1, con=cnx)

In [88]:
df1.head()

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


## Q2. No. of facilities that do not charge a fee to members

In [89]:
query2 = 'SELECT COUNT(DISTINCT(name)) AS cnt FROM Facilities WHERE membercost = 0'

In [90]:
df2 = pd.read_sql(query2, con=cnx)

In [91]:
df2.head()

,cnt
0,4


## Q3. Facilities that charge a fee to members where the fee is less than 20% of the facility's monthly maintenance cost

In [92]:
query3 = '''SELECT facid, name, membercost, monthlymaintenance 
FROM Facilities
WHERE membercost < 0.2 * monthlymaintenance'''

In [93]:
df3 = pd.read_sql(query3, con=cnx)

In [94]:
df3.head()

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000


## Q4. Details of facilities with ID 1 and 5

In [95]:
query4 = 'SELECT * FROM Facilities WHERE facid IN (1, 5)'

In [96]:
df4 = pd.read_sql(query4, con=cnx)

In [97]:
df4.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


## Q5. Facilities with 'cheap' (monthly maintenance cost <= 100) or 'expensive' (monthly maintenance cost > 100) label

In [98]:
query5 = '''SELECT name, monthlymaintenance, IF(monthlymaintenance > 100, 'expensive', 'cheap') AS label 
FROM Facilities'''

In [99]:
df5 = pd.read_sql(query5, con=cnx)

In [100]:
df5.head()

,name,monthlymaintenance,label
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive


## Q6. First and last name of the last member(s) who signed up

In [101]:
query6 = '''SELECT firstname, surname, joindate 
FROM Members
WHERE joindate=(SELECT MAX(joindate) FROM Members)'''

In [102]:
df6 = pd.read_sql(query6, con=cnx)

In [103]:
df6.head()

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45


## Q7. Members who have used a tennis court

In [104]:
query7 = '''SELECT t1.name AS court_name, CONCAT(m.firstname, ' ', m.surname) AS member_name
FROM 
(SELECT DISTINCT b.memid, f.facid, f.name
FROM Bookings b JOIN Facilities f 
ON b.facid=f.facid AND name LIKE '%Tennis Court%') t1 JOIN Members m 
ON t1.memid=m.memid
ORDER BY member_name'''

In [105]:
df7 = pd.read_sql(query7, con=cnx)

In [106]:
df7.head()

,court_name,member_name
0,Tennis Court 2,Anne Baker
1,Tennis Court 1,Anne Baker
2,Tennis Court 2,Burton Tracy
3,Tennis Court 1,Burton Tracy
4,Tennis Court 1,Charles Owen


## Q8. Bookings on 2012-09-14 which will cost the member/ guest more than 30

In [107]:
query8 = '''SELECT f.name, CONCAT(m.firstname, ' ', m.surname) AS member_name, 
IF(b.memid=0, f.guestcost*b.slots, f.membercost*b.slots) AS cost
FROM Bookings b, Facilities f, Members m
WHERE DATE(b.starttime)='2012-09-14' 
AND b.facid=f.facid AND b.memid=m.memid 
AND IF(b.memid=0, f.guestcost*b.slots>30, f.membercost*b.slots>30)
ORDER BY cost DESC'''

In [108]:
df8 = pd.read_sql(query8, con=cnx)

In [109]:
df8.head()

,name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0


## Q9. Same result using subquery

In [110]:
query9 = '''SELECT * 
FROM
(SELECT t1.name AS facility_name, CONCAT(m.firstname, ' ', m.surname) AS member_name, 
IF(t1.memid=0, guestcost*slots, membercost*slots) AS cost 
FROM
(SELECT name, memid, membercost, guestcost, slots 
FROM Bookings b JOIN Facilities f 
ON b.facid=f.facid AND DATE(b.starttime)='2012-09-14') t1
JOIN Members m ON t1.memid=m.memid) t2
WHERE cost>30
ORDER BY cost DESC'''

In [111]:
df9 = pd.read_sql(query9, con=cnx)

In [112]:
df9.head()

,facility_name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0


## Q10. Facilities with a total revenue less than 1000

In [113]:
query10 = '''SELECT name AS facility_name, 
SUM(IF(memid=0, guestcost*slots, membercost*slots)) AS total_revenue 
FROM Bookings b JOIN Facilities f 
ON b.facid=f.facid
GROUP BY name
HAVING total_revenue<1000
ORDER BY total_revenue'''

In [114]:
df10 = pd.read_sql(query10, con=cnx)

In [115]:
df10.head()

,facility_name,total_revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0
